In [1]:
# !which python

In [2]:
# !pip install huggingface-hub

# Hugging Face debug notebook (ask-my-health)

This notebook is for debugging Hugging Face connectivity *outside* the Streamlit app.

## What it does

1. Confirms your `HF_TOKEN` is valid (via `huggingface.co/api/whoami-v2`).
2. Probes a few likely Hugging Face Router endpoints for a **single model**.
3. Sends a **text-to-SQL prompt** and prints:
   - HTTP status code
   - response body (truncated)
   - parsed JSON (if any)

## Setup

Set your token in the same environment where you run Jupyter:

```bash
export HF_TOKEN="hf_..."
```



In [ ]:
import os
from huggingface_hub import InferenceClient
HF_TOKEN = "hf_..."

client = InferenceClient(
    api_key=HF_TOKEN,
)

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3.2:novita",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is **Paris**.', reasoning=None, tool_call_id=None, tool_calls=None)


In [12]:
schema = "daily_steps(date DATE, steps BIGINT)"
question = "How many steps did I walk this month?"

prompt = f"""
You are an expert at writing DuckDB SQL.

Return ONLY a single SQL query (no explanations, no markdown).
Rules:
- Must be a single SELECT statement.
- Query only the table {schema}.
- Do not use any other tables.
- If the question asks for a single number, return exactly one row with column alias answer.

Question: {question}
SQL:
""".strip()

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3.2:novita",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)

print(completion.choices[0].message.content)


SELECT COALESCE(SUM(steps), 0) AS answer FROM daily_steps WHERE strftime(date, '%Y-%m') = strftime(CURRENT_DATE, '%Y-%m');
